# Groupy By

In [4]:
import pandas as pd
import numpy as np

## 메카닉

In [5]:
df = pd.DataFrame(
    {'key1' : ['a', 'a', 'b', 'b', 'a'],
     'key2' : ['one', 'two', 'one', 'two', 'one'],
     'data1' : np.random.randn(5),
     'data2' : np.random.randn(5)
    }
)

df

,key1,key2,data1,data2
0,a,one,-0.270269,0.411422
1,a,two,0.585400,-1.227115
2,b,one,-0.569074,2.202084
3,b,two,-1.786225,0.661759
4,a,one,0.346329,-0.302728


### 하나의 값에 대해서만 group by

In [7]:
# data1에 대해 groupby 메서드를 호출하고 key1 컬럼을 넘기는 것
grouped = df['data1'].groupby(df['key1'])

In [8]:
grouped

In [10]:
grouped.sum()

key1
a    0.661460
b   -2.355299
Name: data1, dtype: float64

In [11]:
grouped.mean()

key1
a    0.220487
b   -1.177649
Name: data1, dtype: float64

### 여러 개의 값에 대해서만 group by

In [12]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [13]:
means

key1  key2
a     one     0.038030
      two     0.585400
b     one    -0.569074
      two    -1.786225
Name: data1, dtype: float64

In [14]:
means.unstack()

key2,one,two
key1,,
a,0.038030,0.585400
b,-0.569074,-1.786225


### 

In [15]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [18]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [19]:
df['data1'].groupby([states,years]).mean()

California  2005    0.585400
            2006   -0.569074
Ohio        2005   -1.028247
            2006    0.346329
Name: data1, dtype: float64

In [21]:
# 한 그룹으로 묶을 정보는 같은 DataFrame안에서 찾고, 이 경우 컬럼 이름을 넘겨서 그룹 색인으로 이용 가능
df.groupby('key1').mean()

# key2값은 숫자 데이터가 아니므로 성가신 컬럼(nuisance column)으로 계산에서 제외시킴

,data1,data2
key1,,
a,0.220487,-0.372807
b,-1.177649,1.431921


In [22]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.038030  0.054347
     two   0.585400 -1.227115
b    one  -0.569074  2.202084
     two  -1.786225  0.661759

In [23]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## 그룹 간 순회하기

In [26]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.270269  0.411422
1    a  two  0.585400 -1.227115
4    a  one  0.346329 -0.302728
b
  key1 key2     data1     data2
2    b  one -0.569074  2.202084
3    b  two -1.786225  0.661759


In [27]:
# 색인 여러 개
for (k1, k2), group in df.groupby([df['key1'], df['key2']]):
    print(k1, k2)
    print(group)

a one
  key1 key2     data1     data2
0    a  one -0.270269  0.411422
4    a  one  0.346329 -0.302728
a two
  key1 key2   data1     data2
1    a  two  0.5854 -1.227115
b one
  key1 key2     data1     data2
2    b  one -0.569074  2.202084
b two
  key1 key2     data1     data2
3    b  two -1.786225  0.661759


In [28]:
# dict로 그룹별 데이터를 사전형으로 바꾸기
pices = dict(list(df.groupby('key1')))

In [29]:
pices['b']

,key1,key2,data1,data2
2,b,one,-0.569074,2.202084
3,b,two,-1.786225,0.661759


In [31]:
# groupby 메서드는 기본적으로 axis=0에 대해 그룹을 만든다
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [33]:
# axis가 아니라 dtype에 따라 그룹으로 묶을 수도 있다
grouped = df.groupby(df.dtypes, axis=1)

In [34]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.270269  0.411422
1  0.585400 -1.227115
2 -0.569074  2.202084
3 -1.786225  0.661759
4  0.346329 -0.302728
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


## 컬럼이나 컬럼의 일부만 선택하기

In [36]:
df.groupby('key1')['data1']
df.groupby('key2')['data2']

df['data1'].groupby(df['key1'])
df['data2'].groupby(df['key2'])

In [37]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.054347
     two  -1.227115
b    one   2.202084
     two   0.661759

In [38]:
df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     one     0.054347
      two    -1.227115
b     one     2.202084
      two     0.661759
Name: data2, dtype: float64

##### 색인으로 얻은 객체는 groupby 메서드에 리스트나 배열을 넘겼을 경우 DataFrameGroupBy객체가 되고, 
##### 단일 값으로 하나의 컬럼 이름만 넘겼을 경우 SeriesGroupBy 객체가 된다

In [39]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

In [40]:
s_grouped

In [41]:
s_grouped.mean()

key1  key2
a     one     0.054347
      two    -1.227115
b     one     2.202084
      two     0.661759
Name: data2, dtype: float64

In [43]:
s_grouped = df.groupby(['key1', 'key2'])[['data2']]
s_grouped

## 사전과 series에서 그룹핑하기

In [47]:
people = pd.DataFrame(np.random.randn(5,5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [48]:
people.iloc[2:3, [1,2]] = np.nan

In [49]:
people

,a,b,c,d,e
Joe,-1.168460,-0.425540,-1.213849,1.300480,0.377211
Steve,1.566527,-0.023816,-0.257520,0.800470,0.406363
Wes,0.691639,NaN,NaN,-0.151330,-0.063658
Jim,-0.374173,-0.144850,0.038700,-0.576541,-1.820345
Travis,0.499774,-0.466440,0.180513,-0.862594,-0.446161


In [50]:
mapping = {'a' : 'red', 'b' : 'blue', 'c' : 'blue', 'd' : 'blue', 'e' : 'red', 'f' : 'orange'}

In [51]:
by_column = people.groupby(mapping, axis=1) # 사용하지 않는 그룹 키도 문제가 없음을 보여ㅛ주기 위해서 f도 포함

In [52]:
by_column.sum()

,blue,red
Joe,-0.338910,-0.791249
Steve,0.519134,1.972890
Wes,-0.151330,0.627981
Jim,-0.682691,-2.194518
Travis,-1.148521,0.053613


In [53]:
map_series = pd.Series(mapping)
map_series

a       red
b      blue
c      blue
d      blue
e       red
f    orange
dtype: object

In [54]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,3,2
Steve,3,2
Wes,1,2
Jim,3,2
Travis,3,2


## 함수로 그룹핑하기

In [55]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.850995,-0.570390,-1.175149,0.572609,-1.506792
5,1.566527,-0.023816,-0.257520,0.800470,0.406363
6,0.499774,-0.466440,0.180513,-0.862594,-0.446161


In [56]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [58]:
people.groupby([len, key_list]).min() #그룹 색인으로 넘긴 함수 ex.len은 색인값

a         b         c         d         e
3 one -1.168460 -0.425540 -1.213849 -0.151330 -0.063658
  two -0.374173 -0.144850  0.038700 -0.576541 -1.820345
5 one  1.566527 -0.023816 -0.257520  0.800470  0.406363
6 two  0.499774 -0.466440  0.180513 -0.862594 -0.446161

## 색인 단계로 그룹핑하기

In [59]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]], names=['cty', 'tenor'])

In [60]:
hier_df = pd.DataFrame(np.random.randn(4,5), columns=columns)

In [62]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.261556 -0.681838  0.019017 -0.057613 -1.326061
1     -1.146675 -0.656783 -0.535999 -0.568725  0.622253
2     -0.973479  0.086307 -0.780585 -0.536992 -0.110211
3     -0.566609  0.359996  0.411936 -0.144228 -0.438469

In [67]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [68]:
hier_df.groupby(level='tenor', axis=1).count()

tenor,1,3,5
0,2,2,1
1,2,2,1
2,2,2,1
3,2,2,1


## 데이터 집계 

##### 배열로부터 스칼라값 만들어내는 데이터 변환 작업

In [69]:
# 최적화된 groupby 메서드 외에도 quantitle 메서드처럼 group by만을 위해 구현된 것은 아닌 Series 메서드 사용 가능
# 내부적으로 groupby는 series를 효과적으로 잘게 자르고 각 조각에 대해 piece.quantitle(0,9) 호출하여 결과들을 하나의 객체로 합쳐서 반환

df

,key1,key2,data1,data2
0,a,one,-0.270269,0.411422
1,a,two,0.585400,-1.227115
2,b,one,-0.569074,2.202084
3,b,two,-1.786225,0.661759
4,a,one,0.346329,-0.302728


In [71]:
grouped = df.groupby('key1')

In [74]:
grouped['data1'].quantile(0.9)

key1
a    0.537586
b   -0.690789
Name: data1, dtype: float64

In [96]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [97]:
grouped.agg(peak_to_peak)

/var/folders/g6/3zj7s36x40v_dqcjss6yzt3w0000gn/T/ipykernel_11872/4193407341.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  grouped.agg(peak_to_peak)


total_bill   tip  size   tip_pct
day  smoker                                  
Fri  No           10.29  2.00     1  0.067349
     Yes          34.42  3.73     3  0.159925
Sat  No           41.08  8.00     3  0.235193
     Yes          47.74  9.00     4  0.290095
Sun  No           39.40  4.99     4  0.193226
     Yes          38.10  5.00     3  0.644685
Thur No           33.68  5.45     5  0.193350
     Yes          32.77  3.00     2  0.151240

In [80]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.220487  0.441497 -0.270269  0.038030  0.346329  0.465865   
b      2.0 -1.177649  0.860656 -1.786225 -1.481937 -1.177649 -0.873362   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.585400   3.0 -0.372807  0.821513 -1.227115 -0.764921 -0.302728   
b    -0.569074   2.0  1.431921  1.089174  0.661759  1.046840  1.431921   

                          
           75%       max  
key1                      
a     0.054347  0.411422  
b     1.817003  2.202084

##  컬럼에 여러 가지 함수 적용하기

In [81]:
tips = pd.read_csv('examples/tips.csv')

In [82]:
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [83]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [84]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [85]:
grouped = tips.groupby(['day','smoker'])

In [86]:
grouped_pct = grouped['tip_pct']

In [88]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [98]:
# 함수 목록을 넘기면 함수 이름을 컬럼으로 하는 DataFrame을 얻게 된다

grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [99]:
# 함수 이름을 컬럼으로 하는 대신 이름과 함수가 담긴 튜플의 리스트를 넘기면 튜플의 첫번재 원소가 컬럼 이름으로 사용된다

grouped_pct.agg([('foo', 'mean'),('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [100]:
# 컬럼마다 다른 함수를 적용하거나 여러 개의 함수를 모든 컬럼에 적용할 수 있다

functions = ['count', 'mean', 'max']

In [101]:
result = grouped['tip_pct', 'total_bill'].agg(functions)

/var/folders/g6/3zj7s36x40v_dqcjss6yzt3w0000gn/T/ipykernel_11872/1540724723.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(functions)


In [102]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [103]:
# 이는 각 컬럼을 따로 계산한 다음 concat 메서드로 keys 인자를 컬럼 이름으로 넘겨서 이어붙인 것과 동일함

result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [106]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [107]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

/var/folders/g6/3zj7s36x40v_dqcjss6yzt3w0000gn/T/ipykernel_11872/514018043.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct', 'total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [108]:
# 컬럼마다 다른 함수를 적용하고 싶다면 agg 메서드에 컬럼 이름에 대응하는 함수가 들어가있는 사전을 넘기면 됨

grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [109]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'], 'size' : 'sum'})
# 단 하나의 컬럼에라도 여러 개의 함수가 적용되었다면 DataFrame은 계층적인 컬럼을 가지게 된다

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

## 색인되지 않은 형태로 집계된 데이터 반환하기

In [110]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## Apply : 일반적인 분리-적용-병합

In [111]:
def top(df, n=5, column = 'tip_pct'):
    return df.sort_values(by=column)[-n:]

In [112]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [113]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [115]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [116]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [117]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [118]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [ ]:
# descirbe 같은 메서드를 호출하면 GroupBy 내부적으로 다음과 같은 단계를 수행

f = lambda x: x.describe()
grouped.apply(f)

In [119]:
# 그룹 색인 생략하기 : group_keys=False
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


## 변위치 분석과 버킷 분석

In [121]:
frame = pd.DataFrame(
    {'data1' : np.random.randn(1000),
     'data2' : np.random.randn(1000)
    }
)

In [123]:
quartiles = pd.cut(frame.data1, 4)


In [125]:
quartiles[:10]

0     (-1.479, 0.115]
1    (-3.079, -1.479]
2      (0.115, 1.709]
3      (0.115, 1.709]
4      (0.115, 1.709]
5     (-1.479, 0.115]
6     (-1.479, 0.115]
7      (1.709, 3.304]
8      (0.115, 1.709]
9     (-1.479, 0.115]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.079, -1.479] < (-1.479, 0.115] < (0.115, 1.709] < (1.709, 3.304]]

In [126]:
def get_status(group):
    return {'min' : group.min(), 'max': group.max,
               'count': group.count(), 'mean' : group.mean()}

In [128]:
grouped = frame.data2.groupby(quartiles)

In [129]:
grouped.apply(get_status).unstack()

,min,max,count,mean
data1,,,,
"(-3.079, -1.479]",-2.107717,<bound method NDFrame._add_numeric_operations....,62,0.246161
"(-1.479, 0.115]",-3.951571,<bound method NDFrame._add_numeric_operations....,474,-0.083485
"(0.115, 1.709]",-3.354937,<bound method NDFrame._add_numeric_operations....,432,-0.090552
"(1.709, 3.304]",-1.417914,<bound method NDFrame._add_numeric_operations....,32,0.238284


In [131]:
grouping = pd.qcut(frame.data1, 10, labels=False)

In [132]:
grouped = frame.data2.groupby(grouping)

In [133]:
grouped.apply(get_status).unstack()

,min,max,count,mean
data1,,,,
0,-3.146171,<bound method NDFrame._add_numeric_operations....,100,0.211538
1,-2.394847,<bound method NDFrame._add_numeric_operations....,100,-0.090868
2,-2.962486,<bound method NDFrame._add_numeric_operations....,100,-0.112668
3,-3.951571,<bound method NDFrame._add_numeric_operations....,100,-0.106403
4,-3.669038,<bound method NDFrame._add_numeric_operations....,100,-0.105036
5,-2.073157,<bound method NDFrame._add_numeric_operations....,100,-0.100325
6,-2.783711,<bound method NDFrame._add_numeric_operations....,100,-0.164726
7,-3.117722,<bound method NDFrame._add_numeric_operations....,100,0.001517
8,-3.354937,<bound method NDFrame._add_numeric_operations....,100,-0.123344


## 예제 : 그룹에 따른 값으로 결측치 채우기

In [134]:
s = pd.Series(np.random.randn(6))

In [135]:
s[::2] = np.nan

In [136]:
s

0         NaN
1    0.116647
2         NaN
3    1.572713
4         NaN
5   -1.103780
dtype: float64

In [137]:
s.fillna(s.mean())

0    0.195193
1    0.116647
2    0.195193
3    1.572713
4    0.195193
5   -1.103780
dtype: float64

In [149]:
# 그룹별로 채워넣고 싶은 값이 다를 때

states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']

In [150]:
group_key = ['East'] * 4 + ['West'] * 4

In [151]:
data = pd.Series(np.random.randn(8), index=states)

In [153]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [154]:
data

Ohio          0.064219
New York     -1.515666
Vermont            NaN
Florida       0.638589
Oregon       -0.910653
Nevada             NaN
California    0.877041
Idaho              NaN
dtype: float64

In [155]:
fill_mean = lambda g: g.fillna(g.mean())

In [157]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.064219
New York     -1.515666
Vermont      -0.270953
Florida       0.638589
Oregon       -0.910653
Nevada       -0.016806
California    0.877041
Idaho        -0.016806
dtype: float64

In [158]:
# 그룹에 따라 미리 정의된 다른 값을 채워넣어야 할 때

fill_values = {'East' : 0.5, 'West' : -1}

In [159]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [160]:
data.groupby(group_key).apply(fill_func)

Ohio          0.064219
New York     -1.515666
Vermont       0.500000
Florida       0.638589
Oregon       -0.910653
Nevada       -1.000000
California    0.877041
Idaho        -1.000000
dtype: float64

## 랜덤 표본과 순열

In [163]:
# 하트, 스페이드, 클럽, 다이아몬ㄴ드
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1,11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []

for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)


In [164]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [166]:
def draw(deck, n=5):
    return deck.sample(n)

In [167]:
draw(deck)

2S    2
AC    1
AH    1
8D    8
5H    5
dtype: int64

In [168]:
get_suit = lambda card: card[-1]

In [169]:
deck.groupby(get_suit).apply(draw, n=2)

C  2C      2
   10C    10
D  10D    10
   7D      7
H  2H      2
   3H      3
S  9S      9
   JS     10
dtype: int64

In [170]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

7C      7
JC     10
10D    10
3D      3
4H      4
7H      7
JS     10
9S      9
dtype: int64

## 예제 : 그룹 가중 평균과 상관관계

In [174]:
df = pd.DataFrame({
    'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
    'data': np.random.randn(8),
    'weights':np.random.rand(8)
})

In [175]:
df

,category,data,weights
0,a,1.297035,0.534868
1,a,-1.002056,0.613710
2,a,-0.792034,0.533424
3,a,-1.245477,0.623282
4,b,0.443190,0.110438
5,b,1.375922,0.546410
6,b,0.202514,0.373926
7,b,-0.127183,0.156959


In [176]:
grouped = df.groupby('category')

In [177]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

In [179]:
grouped.apply(get_wavg)

category
a   -0.485842
b    0.721143
dtype: float64

## 더 복잡한 예제(주식과 S&P 500 지수)

In [181]:
close_px = pd.read_csv('examples/stock_px.csv', parse_dates=True, index_col=0)

In [182]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [183]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [184]:
spx_corr = lambda x:x.corrwith(x['SPX'])

In [185]:
rets = close_px.pct_change().dropna()

In [186]:
get_year = lambda x: x.year

In [187]:
by_year = rets.groupby(get_year)

In [188]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [189]:
# 두 컬럼의 상관관계

by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64